# Instructions
**1. Read the course schedule into a DataFrame**

the frame should have the following columns:
- Number-Section: the course number and section number

Use your discretion to deal with issues encountered here (for example, some issues may include courses listed with two different course numbers!), but document what you've done and give some rationale for your methodology

⚠️ you may encounter an invisible space character (a zero width space) in the course number depending on how you extract the text (view the markup in Chrome's web inspector tools or try printing it out in Python)

the easiest way to deal with this is to replace it with emtpy string (assuming s contains the zero width space): s.replace('\u200b', '')
- Name: the name of the course
- Instructor: the name of the professor

Again, there may be issues here, such as multiple instructors; use your discretion, but describe what you've done and why
- Time: the day(s) and time(s) the course meets
(once you read in the data, you'll add a couple of rows)

In [41]:
import requests
from bs4 import BeautifulSoup
import re


In [252]:
import pandas as pd
import numpy as np

def html_to_table_schedule(link):
    with open(link, 'r') as f:
        dom = BeautifulSoup(f.read(), 'html')
    dom = dom.find_all("ul", class_="schedule-listing")[0]
    course_lists = dom.find_all('li')
    id_list = []
    name_list = []
    instructor_list = []
    time_list = []
    for course in course_lists:
        try:
            id = course['id']
        except:
            id = 'Not Found.'
        id_list.append(id.upper())

        try:
            name = course.find_all('a', class_ = 'expand')[0].text.replace('\n', '').strip(' ')
        except:
            name = 'Not Found'
        name_list.append(name)

        try:
            instructor = course.find_all('span', class_ = 'col-xs-12 col-sm-2')[0].text.replace('\n', '').strip(' ')
        except:
            instructor = 'Not Found'
        instructor_list.append(instructor)

        try:
            time = course.find_all('span', class_ ="col-xs-12 col-sm-2")[1].text
        except:
            time = 'Not Found'
        time_list.append(time)

    table = pd.DataFrame(np.array([id_list, name_list, instructor_list, time_list]).T, columns = ['Number-Section', 'Name', 'Instructor', 'Time'])
    return table

schedule_link = 'course_schedule.html'
schedule_table = html_to_table_schedule(schedule_link)
schedule_table

,Number-Section,Name,Instructor,Time
0,CSCI-GA1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM
1,CSCI-GA1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM
2,CSCI-GA1170-003,Fundamental Algorithms,Richard Cole,W 4:55-6:55PM
3,CSCI-GA1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM
4,CSCI-GA1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM
...,...,...,...,...
155,CSCI-UA0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM
156,CSCI-UA0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM
157,CSCI-UA0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM
158,CSCI-UA0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM


- once you've read in your data, break apart the Number-Section column into two separate columns: Number and Section

    - Number is something like CSCI-UA.0480

    - Section is something like 001
    - try to use regular expressions with groups to do this
        - the str accessor method to use is extract
        - check out the end of the regex slides or the official pandas docs
    - show:
        - info to show the data types and counts
        - the first 5 rows
        - the last 5 rows
        - a random sampling of 5 rows

In [253]:
schedule_table['Number'] = schedule_table['Number-Section'].str.extract(r'(.*)-')
schedule_table['Section'] = schedule_table['Number-Section'].str.extract(r'(\d+)')
schedule_table

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,CSCI-GA1170,1170
1,CSCI-GA1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,CSCI-GA1170,1170
2,CSCI-GA1170-003,Fundamental Algorithms,Richard Cole,W 4:55-6:55PM,CSCI-GA1170,1170
3,CSCI-GA1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,CSCI-GA1170,1170
4,CSCI-GA1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,CSCI-GA1170,1170
...,...,...,...,...,...,...
155,CSCI-UA0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,CSCI-UA0480,0480
156,CSCI-UA0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA0480,0480
157,CSCI-UA0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,CSCI-UA0480,0480
158,CSCI-UA0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA0480,0480


In [254]:
# info to show the data types and counts
schedule_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  160 non-null    object
 1   Name            160 non-null    object
 2   Instructor      160 non-null    object
 3   Time            160 non-null    object
 4   Number          160 non-null    object
 5   Section         160 non-null    object
dtypes: object(6)
memory usage: 7.6+ KB


In [255]:
# the first 5 rows
schedule_table.head(5)

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,CSCI-GA1170,1170
1,CSCI-GA1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,CSCI-GA1170,1170
2,CSCI-GA1170-003,Fundamental Algorithms,Richard Cole,W 4:55-6:55PM,CSCI-GA1170,1170
3,CSCI-GA1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,CSCI-GA1170,1170
4,CSCI-GA1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,CSCI-GA1170,1170


In [256]:
# the last 5 rows
schedule_table.tail(5)

,Number-Section,Name,Instructor,Time,Number,Section
155,CSCI-UA0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,CSCI-UA0480,0480
156,CSCI-UA0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA0480,0480
157,CSCI-UA0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,CSCI-UA0480,0480
158,CSCI-UA0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA0480,0480
159,CSCI-UA0480-071,Special Topics: ...,Evan Korth,TR 3:30-4:45PM,CSCI-UA0480,0480


In [257]:
# a random sampling of 5 rows
schedule_table.sample(5)

,Number-Section,Name,Instructor,Time,Number,Section
19,CSCI-GA2433-001,Database Systems,Jean-Claude Franchitti,R 4:55-6:55PM,CSCI-GA2433,2433
5,CSCI-GA1170-007,Fundamental Algorithms Recitation,Nick Lourie,F 4:55-5:45PM,CSCI-GA1170,1170
128,CSCI-UA0201-061,Computer Systems Organization - Recitation,Samvid Zare,T 4:55-6:10PM,CSCI-UA0201,0201
113,CSCI-UA0102-061,Data Structures - Recitation,Harshitha Raghavan Devarajan,R 11:00-12:15PM,CSCI-UA0102,0102
72,CSCI-GA3870-001,Internship In Computer Science MS*,STAFF,-,CSCI-GA3870,3870


**2. Read the course catalog into a DataFrame**
- the frame should have the following columns:
    - Number: the course number
    - Prereqs: a text description of the prerequisites
    - Points: the number of credits
- here's example of a DataFrame with some course catalog rows (again, the data is from another semester)

In [258]:

def html_to_table_catalog(link):
    with open(link, 'r') as f:
        dom = BeautifulSoup(f.read(), 'html')
    dom = dom.select("ul", class_ = 'courses-listing')[2]
    course_list = dom.select('li')
    id_list = []
    prereqs_list = []
    points_list = []
    for course in course_list:
        try:
            id = course['id'].upper()
        except:
            id = 'Not Found'
        id_list.append(id)
        
        try:
            prereq = course.find_all('p', class_ = 'bold')[-1].text.replace('Prerequisites: ', '')
        except:
            prereq = 'Not Found'
        prereqs_list.append(prereq)

        try:
            s = course.find_all('p', class_ = 'bold')[1].text.replace('\n', '').strip(' ')
            match = re.search(r'(\d) Points', s)
            points = int(match[1])
        except:
            points = np.nan
        points_list.append(points)
        
        catalog_table = pd.DataFrame(np.array([id_list, prereqs_list, points_list ]).T, columns = ['Number', 'Prereqs', 'Points'])
    return catalog_table
catalog_link = 'course_catalog.html'
catalog_table = html_to_table_catalog(catalog_link)
catalog_table

,Number,Prereqs,Points
0,CSCI-GA1133,None,4
1,CSCI-GA1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA1170,At least one year of experience with a high-le...,3
3,CSCI-GA1180,None,3
4,CSCI-GA2110,Students taking this class should already have...,3
...,...,...,...
91,CSCI-UA0897,Restricted to declared computer science majors...,4
92,CSCI-UA0898,Restricted to declared computer science majors...,4
93,CSCI-UA0997,Permission of the department. Does not satisfy...,4
94,CSCI-UA0998,Permission of the department. Does not satisfy...,4


- show:
    - info to show the data types and counts
    - the first 5 rows
    - the last 5 rows
    - a random sampling of 5 rows
- use a similar parsing strategy as above to read in this DataFrame  

In [259]:
# infor to show the data types and counts
catalog_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   96 non-null     object
 1   Prereqs  96 non-null     object
 2   Points   96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [260]:
# the first 5 rows
catalog_table.head(5)

,Number,Prereqs,Points
0,CSCI-GA1133,None,4
1,CSCI-GA1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA1170,At least one year of experience with a high-le...,3
3,CSCI-GA1180,None,3
4,CSCI-GA2110,Students taking this class should already have...,3


In [261]:
# the last 5 rows
catalog_table.tail(5)

,Number,Prereqs,Points
91,CSCI-UA0897,Restricted to declared computer science majors...,4
92,CSCI-UA0898,Restricted to declared computer science majors...,4
93,CSCI-UA0997,Permission of the department. Does not satisfy...,4
94,CSCI-UA0998,Permission of the department. Does not satisfy...,4
95,FRSEM-UA0597,"Some programming experience in Python, Java, J...",4


In [262]:
# the last 5 rows
catalog_table.sample(5)

,Number,Prereqs,Points
50,CSCI-GA3210,Strong mathematical background,3
21,CSCI-GA2437,"Prerequisites include experience with Hadoop, ...",3
57,CSCI-GA3840,Approval of a faculty adviser and the Director...,6
86,CSCI-UA0479,Data Structures (CSCI-UA 102). Students that c...,4
92,CSCI-UA0898,Restricted to declared computer science majors...,4


**3. Put together both DataFrames**
- create a new DataFrame by….
- finding a way to show all scheduled classes in the semester along with their points and prereqs
- only show the following columns, in this order:
    - Number: course number
    - Name: course name
    - Instructor: professor's name
    - Time: meeting time
    - Prereqs: course prerequisites
    - Points: number of credits
- hints:
    - use pd.merge to do this
    - how=left will keep all rows in the first DataFrame

In [263]:
schedule_table

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,CSCI-GA1170,1170
1,CSCI-GA1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,CSCI-GA1170,1170
2,CSCI-GA1170-003,Fundamental Algorithms,Richard Cole,W 4:55-6:55PM,CSCI-GA1170,1170
3,CSCI-GA1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,CSCI-GA1170,1170
4,CSCI-GA1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,CSCI-GA1170,1170
...,...,...,...,...,...,...
155,CSCI-UA0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,CSCI-UA0480,0480
156,CSCI-UA0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA0480,0480
157,CSCI-UA0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,CSCI-UA0480,0480
158,CSCI-UA0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA0480,0480


In [264]:
catalog_table

,Number,Prereqs,Points
0,CSCI-GA1133,None,4
1,CSCI-GA1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA1170,At least one year of experience with a high-le...,3
3,CSCI-GA1180,None,3
4,CSCI-GA2110,Students taking this class should already have...,3
...,...,...,...
91,CSCI-UA0897,Restricted to declared computer science majors...,4
92,CSCI-UA0898,Restricted to declared computer science majors...,4
93,CSCI-UA0997,Permission of the department. Does not satisfy...,4
94,CSCI-UA0998,Permission of the department. Does not satisfy...,4


In [268]:
combined = pd.merge(schedule_table, catalog_table, on = 'Number', how = 'left')
combined = combined[['Number', 'Name', 'Instructor', 'Time', 'Prereqs', 'Points']]
combined

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA1170,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,At least one year of experience with a high-le...,3
1,CSCI-GA1170,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,At least one year of experience with a high-le...,3
2,CSCI-GA1170,Fundamental Algorithms,Richard Cole,W 4:55-6:55PM,At least one year of experience with a high-le...,3
3,CSCI-GA1170,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,At least one year of experience with a high-le...,3
4,CSCI-GA1170,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,At least one year of experience with a high-le...,3
...,...,...,...,...,...,...
155,CSCI-UA0480,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,Topics determine prerequisites.,4
156,CSCI-UA0480,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,Topics determine prerequisites.,4
157,CSCI-UA0480,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,Topics determine prerequisites.,4
158,CSCI-UA0480,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,Topics determine prerequisites.,4


**4. Conclusion**
- did you spot any anomalies, discrepancies, or unexpected data or relationships between data?
- if so, in a markdown cell, describe any problem(s) you saw
- additionally, describe how you might fix them (or if you already fixed them!)
- lastly, based on the resulting DataFrame, describe the behavior of how=left on these particular DataFrames
- if you need to see all rows, use pd.set_option('display.max_rows', 200)
***

There might be multiple class namse associated with one course number. Also there are duplicates in terms of courses in this new table. Sometimes, several rows refer to the exact same course, but the scraping program and merge create a distince row for each meeting time.


In [281]:
combined[32:38]

,Number,Name,Instructor,Time,Prereqs,Points
32,CSCI-GA2630,Foundations of Networks and Mobile Systems*,Lakshminarayanan Subramanian,T 10:00-11:50AM,None,3
33,CSCI-GA2630,Foundations of Networks and Mobile Systems Lab*,Lakshminarayanan Subramanian,T 9:00-9:50AM,None,3
34,CSCI-GA2630,Foundations of Networks and Mobile Systems*,Lakshminarayanan Subramanian,M 12:30-2:30PM,None,3
35,CSCI-GA2630,Foundations of Networks and Mobile Systems Lab*,Lakshminarayanan Subramanian,W 1:30-2:20PM,None,3
36,CSCI-GA2810,Design and Innovation*,Gary Zamchick,F 10:15-12:15PM,None,3
37,CSCI-GA2810,Design and Innovation Lab*,Gary Zamchick,F 12:30-1:20PM,None,3


I solve the first issue by grouping by Number, class Name, and Instructor. We can see more clearly that a class number may have multiple class names associated with it. 
I solve the second issue by merging the meeting times of the same course(with the same course number, name and instructor).

In [283]:
pd.set_option('display.max_rows', 200)
array_agg = lambda x: '/'.join(x)

merge_time= combined.groupby(['Number', 'Name', 'Instructor']).agg({'Time': array_agg})
merge_time

Time
Number      Name                                               Instructor                                                                         
CSCI-GA1170 Fundamental Algorithms                             Chee Yap                                                              T 4:55-6:55PM
                                                               Richard  Cole                                                         W 4:55-6:55PM
            Fundamental Algorithms Recitation                  Daniel Feldan                                                         R 5:55-6:45PM
                                                               Nick Lourie                                                           F 4:55-5:45PM
                                                               Zachary DeStefano                                                     F 4:55-5:45PM
                                                               Zeming Lin                                                            R 5:55-6:45PM
CSCI-GA1180 Mathematical Techniques For CS Applications        Parijat Dube                                                          M 7:10-9:10PM
CSCI-GA2110 Programming Languages                              Cory Plock                                                            M 4:55-6:55PM
                                                               Thomas Wies                                                           T 4:55-6:55PM
            Programming Languages Recitation                   Elaine Li                                                             F 5:55-6:45PM
                                                               Jahnvi Arya                                                           R 4:55-5:45PM
                                                               Nisarg Patel                                                          F 5:55-6:45PM
                                                               Yashaswi Shah                                                         R 4:55-5:45PM
CSCI-GA2112 Scientific Computing                               Jonathan Goodman                                                      R 5:10-7:00PM
CSCI-GA2250 Operating Systems                                  Hubertus Franke                                                       M 7:10-9:10PM
CSCI-GA2262 Data Communications and Networks                   Jean-Claude Franchitti                                                R 7:10-9:10PM
CSCI-GA2270 Computer Graphics                                  Gizem Kayar                                                           W 7:10-9:10PM
CSCI-GA2271 Computer Vision                                    Rob Fergus                                                            R 7:10-9:10PM
CSCI-GA2420 Numerical Methods I                                Benjamin Peherstorfer                                                 M 5:10-7:00PM
CSCI-GA2433 Database Systems                                   Jean-Claude Franchitti                                                R 4:55-6:55PM
CSCI-GA2434 Advanced Database Systems                          Dennis  Shasha                                                        R 4:55-6:55PM
CSCI-GA2436 Realtime and Big Data Analytics                    Yang Tang                                                             W 7:10-9:10PM
CSCI-GA2437 Big Data Application Development                   Yang Tang                                                             M 7:10-9:10PM
CSCI-GA2560 Artificial Intelligence                            Paul Bethe                                                            M 4:55-6:55PM
CSCI-GA2565 Machine Learning                                   Rajesh Ranganath                                                      T 4:55-6:55PM
CSCI-GA2566 Foundations Of Machine Learning*                   CANCELLED                                                                         -
CSCI-GA2569 Inference and 

In [288]:
print(f'The first table contains: {len(schedule_table.Number.unique())} distinct course numbers and course info.')
print(f'The second table contains: {len(catalog_table.Number.unique())} distinct course numbers and their prerequisites.')


The first table contains: 55 distinct course numbers and course info.
The second table contains: 96 distinct course numbers and their prerequisites.


Merging the two tables with `how = left` ensures that all entries from the first table are kept in the new table, regardless of whether the course number appears in the second table.